In [1]:
#Compute a Photon Transfer Curve using a script that takes pairs of images at increasing exposure times. 

In [2]:
import sys
import asyncio
import logging
import time

import numpy as np

from lsst.ts import salobj


from lsst.ts.standardscripts.auxtel.attcs import ATTCS
from lsst.ts.standardscripts.auxtel.latiss import LATISS

In [3]:
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [4]:
attcs = ATTCS()
latiss = LATISS(attcs.domain) 

Read historical data in 0.01 sec
Read historical data in 0.28 sec
Read historical data in 1.13 sec
Read historical data in 1.39 sec
Read historical data in 2.06 sec
Read historical data in 2.67 sec
Read historical data in 3.02 sec
Read historical data in 4.04 sec
Read historical data in 4.66 sec
Read historical data in 5.38 sec
Read historical data in 6.21 sec


In [5]:
await attcs.enable()
await latiss.enable() 

RemoteTelemetry(ATDome, 0, position) falling behind; read 24 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 14 messages
Gathering settings.
No settings for atmcs.
Using                                                                                                                                 from settingVersions event.
No settings for atptg.
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 15 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 15 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 16 messages
Couldn't get settingVersions event. Using empty settings.
No settings for ataos.
Using current from settingVersions event.
No settings for atpneumatics.
Using                                                                                                                                 from settingVersions event.
No settings for athexapod.
Using current from settingVersions event.
No settings for 

In [6]:
# !!!!!!! STOP !!!!!! 
#If the system is not already in position for flat fields and you would like to use flats as the basis for your PTC,
# execute this cell.  Otherwise, don't bother.
# !!!!! IF DOME IS ALREADY IN FLAT FIELD POSITION, YOU DONT'T NEED TO RUN THIS CELL 
await attcs.prepare_for_flatfield()

Cover state <MirrorCoverState.OPENED: 7>
M1 cover already opened.
Sending command
Stop tracking.
Mount tracking state is 8
In Position: False.
Scheduling check coroutines
process as completed...
Got False
Telescope not in position
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
[Telescope] delta Alt = -018.720 | delta Az= +016.644
[Telescope] delta Alt = -018.528 | delta Az= +016.210
[Telescope] delta Alt = -017.320 | delta Az= +014.397
[Telescope] delta Alt = -014.765 | delta Az= +011.188
[Telescope] delta Alt = -011.170 | delta Az= +007.265
[Telescope] delta Alt = -007.211 | delta Az= +003.613
[Telescope] delta Alt = -003.683 | delta Az= +001.102
[Telescope] delta Alt = -002.298 | delta Az= +000.416
[Telescope] delta Alt = -000.533 | delta Az= +000.001
[Telescope] delta Alt = -000.011 | delta Az= -000.000
[Telescope] delta Alt = +000.000 | delta Az= -000.000
Got True
Waiting for telescope to 

In [7]:
#Select your filter and grating
filter_str = 'empty_1'
grating_str = 'empty_4'

In [10]:
#Define your exposure times. 
#If you are not sure about your exposure times, consider taking a set with only one exposure time and basing your scaling off of that 
# (see the DisplayImage.ipynb for instructions on pulling up the flat field image that you produce). 
exp_times = [0.25 * scaling for scaling in [1.0] + np.arange(2.0, 20.5, 2.0).tolist()]
print ('exposure times are ' + str(exp_times))

exposure times are [0.25, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]


In [11]:
for exp_time in exp_times: 
    PTC_pairs = await latiss.take_flats(exptime=exp_time, nflats=2, filter=filter_str, grating=grating_str)
    print ('The PTC pair data ids for exposure time ' +str(exp_time) + ' are: '+  str(PTC_pairs)) 

Generating group_id
FLAT 0001 - 0002
Waiting for header service LFO before continuing
FLAT 0002 - 0002
Waiting for header service LFO before continuing
The PTC pair data ids for exposure time 0.25 are: [2020022000037 2020022000038]
Generating group_id
FLAT 0001 - 0002
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
FLAT 0002 - 0002
Waiting for header service LFO before continuing
The PTC pair data ids for exposure time 0.5 are: [2020022000039 2020022000040]
Generating group_id
FLAT 0001 - 0002
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages
FLAT 0002 - 0002
Waiting for header service LFO before continuing
The PTC pair data ids for exposure time 1.0 are: [2020022000041 2020022000042]
Generating group_id
FLAT 0001 - 0002
Waiting for header service LFO before continuing
FLAT 0002 - 0002
Waiting for header service LFO before continuing
